# A Simple Sales Agent
https://github.com/microsoft/Trace

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from opto import trace

@trace.model
class Agent:

    def __init__(self, system_prompt):
        self.system_prompt = system_prompt
        self.instruct1 = trace.node("Decide the language", trainable=True)
        self.instruct2 = trace.node("Extract name if it's there", trainable=True)

    def __call__(self, user_query):
        language = trace.operators.call_llm(self.system_prompt,
                                            user_query, self.instruct1)
        en_or_es = self.decide_lang(language)

        user_name = trace.operators.call_llm(self.system_prompt,
                                             user_query, self.instruct2)
        greeting = self.greet(en_or_es, user_name)

        return greeting

    @trace.bundle(trainable=True)
    def decide_lang(self, response):
        """Map the language into a variable"""
        return

    @trace.bundle(trainable=True)
    def greet(self, lang, user_name):
        """Produce a greeting based on the language"""
        greeting = ""
        return f"{greeting}, {user_name}!"

In [4]:
from opto.optimizers import OptoPrime

def feedback_fn(generated_response, gold_label='en'):
    if  gold_label == 'en' and 'Hello' in generated_response:
        return "Correct"
    elif gold_label == 'cn' and '你好，Xiaoming' == generated_response:
        return "Correct"
    else:
        return "Incorrect"

def train():
    epoch = 5
    agent = Agent("You are a sales assistant.")
    optimizer = OptoPrime(agent.parameters())

    for i in range(epoch):
        print(f"Training Epoch {i}")
        try:
            greeting = agent("Xiaoming is a customer from China.")
            feedback = feedback_fn(greeting.data, 'cn')
        except trace.ExecutionError as e:
            greeting = e.exception_node
            feedback, terminal, reward = greeting.data, False, 0

        optimizer.zero_feedback()
        optimizer.backward(greeting, feedback)
        optimizer.step(verbose=True)

        if feedback == 'Correct':
            break

    return agent

agent = train()

Using CustomLLM as the default LLM backend.
Training Epoch 0
Prompt
 
You're tasked to solve a coding/algorithm problem. You will see the instruction, the code, the documentation of each function used in the code, and the feedback about the execution result.

Specifically, a problem will be composed of the following parts:
- #Instruction: the instruction which describes the things you need to do or the question you should answer.
- #Code: the code defined in the problem.
- #Documentation: the documentation of each function used in #Code. The explanation might be incomplete and just contain high-level description. You can use the values in #Others to help infer how those functions work.
- #Variables: the input variables that you can change.
- #Constraints: the constraints or descriptions of the variables in #Variables.
- #Inputs: the values of other inputs to the code, which are not changeable.
- #Others: the intermediate values created through the code execution.
- #Outputs: the result

In [5]:
print(agent("Xiaomi is a customer from China."))

MessageNode: (eval:11, dtype=<class 'str'>, data=您好, Xiaomi!)


In [11]:
for param in agent.parameters():
    print(f"Parameter: {param.name}\n {param.data}")

Parameter: __code:2
 def decide_lang(self, response):
    """Map the language into a variable"""
    if "Chinese" in response or "Mandarin" in response:
        return "Mandarin"
    elif "English" in response:
        return "English"
    elif "Other" in response:
        return "Ask"  # Special handling for ambiguity.
    else:
        return "Unknown"

Parameter: __code:4
 def greet(self, lang, user_name):
    """Produce a greeting based on the language"""
    if lang == "Mandarin":
        greeting = "您好"  # Use a formal greeting format for Mandarin.
    elif lang == "English":
        greeting = "Hello"
    elif lang == "Ask":
        greeting = "What language would you prefer?"
    else:
        greeting = "Hi"
    return f"{greeting}, {user_name}!"

Parameter: str:0
 Decide the language
Parameter: str:1
 Extract only the name provided
